<a href="https://colab.research.google.com/github/wilsonufrj/DatathonONS/blob/main/DatathonONS_3_Edi%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Ideias Iniciais</h1>
<ul>
  <li>Separar os dados globais em subregiões</li>
  <li>Verificar se a relação entre temperatura e cogeração é linear ( Quando tem cogeração a usina é despachavel?)</li>
  <li>Testar combinações dos parametros para melhorar a descretização das subregiões</li>
</ul>

Paramêtros -> Areas Industriais , Areas Residenciais, sim_mmgd, nao_mmgd, sim_cogeracao, nao_cogeracao  


Cogeraçao = Geração não dependente da rede

<h1>Insight sobre os Datasets</h1>
Temos a quantidade de geração de energia de uma distribuidora durante uma semana 
<hr>
Temos 8 distribuidoras e um periodo de tempo de 01-01-2018 até aproximadamente 31-03-2022

<h1>Tarefas</h1>
<ol>
  <li>Saber qual distribuidora distribui pra cada município para agrupar os dados de temperatura por distribuidora. Gerar uma nova estrutura de dados com isso</li>
  <li>Tratar as medições que estão como null</li>
  <li>No inmet, temos 3 medições de temperatura por hora, fazer a média para cada hora para termos um arquivo com uma medição por hora</li>
  <li>Quando o campo estiver vazio, colocar 0</li>
  <li>Plotar os gráficos Carga X Temperatura X Tempo</li>
  <li>Fazer um gráfico para cada distribuidora</li>
</ol>





In [ ]:
#Inicialização no drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Importações das bibliotecas
import pandas as pd
import pprint

In [ ]:
#Caminho
path = "/content/drive/MyDrive/"

In [ ]:
# Definição de cabeçalhos para os DataFrames
Cabecalho_SerieCargas = ["nom_seriehistorica", "din_medicao", "val_itemserieoriginal"]
Cabecalho_Municipios = ["codigo_ibge", "nome", "latitude", "longitude", "capital",
                        "codigo_uf", "siafi_id", "ddd", "fuso_horario"]
Cabecalho_INMET = ['id_estacaometeo',	'id_varmeteo',	'nom_varmeteo',	'nom_longo',	'din_medicao',	'din_inclusaodl',	'val_medicao',	'val_altitude',	'val_latitude',	'val_longitude']                        

In [ ]:
#DataFrames
dataFrameSerieCargas = pd.read_csv(path + 'Datathons/seriesCargaSP.csv', delimiter=';', skiprows = 1, names = Cabecalho_SerieCargas)
dataFrameMunicipios = pd.read_csv(path +'Datathons/Municipios_SP.csv',delimiter=',', skiprows = 1, names = Cabecalho_Municipios)
dataFrameTemperaturaAreaAtuacao = pd.read_csv(path +'Datathons/inmet.csv', delimiter=';')
dataFrameDistribuidoras = pd.read_excel(path +'Datathons/Distribuidoras_Area_Atuacao.xlsx')

In [ ]:
#Média das cargas em todo o período por distribuidora
Distribuidoras = dataFrameSerieCargas["nom_seriehistorica"].unique()
Media_Carga_Distribuidoras = []
for dist in Distribuidoras:
    Serie_Distribuidoras = dataFrameSerieCargas[dataFrameSerieCargas["nom_seriehistorica"] == dist]
    Media = Serie_Distribuidoras["val_itemserieoriginal"].str.replace(',', '').astype(float).mean()
    Media_Carga_Distribuidoras.append(Media/1000000) # a divisão foi feita para os valores aparecerem em MW


In [ ]:
#Estatic Value
distribuidorasMunicipio = {
    'CPFL SANTA CRUZ':[],
    'ELETROPAULO':[],
    'CPFL PIRATININGA':[],
    'CPFL PAULISTA':[],
    'ELEKTRO':[],
    'EDP SP':[],
    'ENERGISA SSE-SP':[]
}


In [ ]:
#Definir os municipios que cada distribuidora trabalha
distribuidoras = dataFrameDistribuidoras['Distribuidora'].unique()

pp = pprint.PrettyPrinter(indent=4)

for i in range(dataFrameDistribuidoras.shape[0]):
  if(dataFrameDistribuidoras['Estado'][i] == 'SP'):  
    distribuidorasMunicipio[dataFrameDistribuidoras['Distribuidora'][i]].append(dataFrameDistribuidoras['Município'][i])

In [ ]:
#Temperatura Bulbo Seco e suas respectivas datas
Data = dataFrameTemperaturaAreaAtuacao["din_medicao"].unique()
Bulbo_Seco = dataFrameTemperaturaAreaAtuacao[dataFrameTemperaturaAreaAtuacao["id_varmeteo"] == "TEM_INS"]
Temperatura_Data = Bulbo_Seco[["nom_longo","val_medicao","din_medicao"]]
Bulbo_Seco.isnull().sum()
Temperatura_Data.head(5)

<h1>Ideias do dia 2</h1>
<ul>
  <li>Fazer um mapa de densidade de carga por distribuidora </li>
  <li>Fazer um mapa de densidade de temperatura por município</li>
  <li>Fazer o grafico temperatura X Carga por distribuidora</li>
  <li>Pensar um modelo de maching learning</li>
</ul>

Tarefas do dia 2
<ul>
  <li>Angelo e Matheus - Fazer um dataframe com a relaçao temperatura e carga consumida </li>
  <li>Wilson  - Pesquisar sobre bibliotecas de mapas</li>
  <li>Gustavo - Fazer o template dos slides</li>
  <li>Implementar um modelo de regressão</li>
</ul>

In [ ]:
#Temperatura_Data=Temperatura_Data.sort_values(by="din_medicao")
#dataFrameSerieCargas=dataFrameSerieCargas.sort_values(by="din_medicao")
#dataFrameFinal=pd.merge_ordered(Temperatura_Data,dataFrameSerieCargas,on='din_medicao')
#dataFrameFinal.head(5)
# TEMPERATURA = []
# CARGA=[]
# DATA=[]
dataFrameSerieCargas.assign('val_medicao')
print(Temperatura_Data)
#for i in range(len(Temperatura_Data['din_medicao'])):

#   for j in range(len(dataFrameSerieCargas['din_medicao'])):
#     a=Temperatura_Data['din_medicao'][i]
#     b=dataFrameSerieCargas['din_medicao'][j]
#     print(a,' ',b)
#     if (a==b):
#       DATA.append(Temperatura_Data['din_medicao'][i])
#       CARGA.append(dataFrameSerieCargas["val_itemserieoriginal"][j])
#       TEMPERATURA.append(Temperatura_Data['val_medicao'][i])
#       break


#dfinal = df1.merge(df2, on="movie_title", how = 'inner')
# print(Temperatura_Data.head(1))
# print(dataFrameSerieCargas.head(1))
# joaninha = Temperatura_Data.set_index("din_medicao").join(dataFrameSerieCargas.set_index("din_medicao"))
# joaninha.head(5)
#print(joaninha)
#df.set_index('key').join(other.set_index('key'))
#dataFrameTemperaturaAreaAtuacao.head(0)
#df.join(other, lsuffix='_caller', rsuffix='_other')